In [1]:

import urllib
import seaborn as sns
import matplotlib.pylab as plt
from bs4 import BeautifulSoup
import bs4
import urllib
import urllib.request
import html.parser
import requests
from requests.exceptions import HTTPError
from socket import error as SocketError
from http.cookiejar import CookieJar

#Original URl

url='https://en.wikipedia.org/wiki/'

#Source URL



source_url = 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Wuhan_coronavirus_outbreak'
with urllib.request.urlopen(source_url) as response:
    soup = BeautifulSoup(response)

#Unclean URLS - As the name suggests - All the URLS with href 

uncleaned_urls = set()
for a_tag in soup.body.find_all('a'):
     if 'href' in a_tag.attrs:
        url = a_tag.attrs['href']
        uncleaned_urls.add(url)
 
 
 #Normalized Urls
 
normalized_urls = set()
for unclean_url in uncleaned_urls:
    
    # First convert the URL to something absolute by "joining"
    # it with the source URL we got it from
    abs_url = urllib.parse.urljoin(source_url, unclean_url)
    
    # Now let's remove the fragment part 
    # the part after the # in the URL
    # We're going to leave in query strings because although they might
    # resolve to the same resource, we might not be sure.
    defrag_url = urllib.parse.urldefrag(abs_url).url
    
    # Now we can add it to our set
    normalized_urls.add(defrag_url)
    
    
 #Filtering for webpage Urls
 
webpage_urls = set()
content_types = set()
for url in normalized_urls:
    try:
        with urllib.request.urlopen(url, None, 5) as request:
            content_types.add(request.headers.get_content_type())
            if request.headers.get_content_type()  == 'text/html':
                webpage_urls.add(url)
    except:
        pass #If there is a 404 error or the like, just go to the next url
        
 # Mapping the same domain urls9from Source URl) with the webpage_urls 
 
same_domain_urls = set()
parsed_source = urllib.parse.urlparse(source_url)
for url in webpage_urls:
    parsed_target = urllib.parse.urlparse(url)
    if parsed_target.netloc == parsed_source.netloc:
        same_domain_urls.add(url)
        
print(len(same_domain_urls))


#Checking if the number of urls decreased from uncleaned to same_domain 

print(len(same_domain_urls))
print(len(webpage_urls))
print(len(normalized_urls))
print(len(uncleaned_urls))
    
import lxml.html
import pandas as pd

# converting the set of same_domain_urls to a Data Frame

dt= pd.DataFrame(same_domain_urls)

#Renaming
dt.rename({0:'Same Domain URLs'}, axis= 1, inplace=True)

#New blank column for Titles

dt["Title"] = ""

#Loop that runs to fetch the title and adds it to the Title column 

for i in range(len(same_domain_urls)):
    url = dt['Same Domain URLs'].iloc[i]
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content, 'lxml')
    dt['Title'].iloc[i] = soup.title.string
    
dt 

#Column Reaarrangement

dt.rename({'Same Domain URLs':'url_target','Title':'page_title_target'}, axis= 1, inplace=True)

dt["url_source"] = ""

dt["url_source"] = "https://en.wikipedia.org/wiki/2019%E2%80%9320_Wuhan_coronavirus_outbreak"

dt["dummy"]= dt["url_source"]

dt["url_source"] = dt["page_title_target"]

dt["page_title_target"] = dt["url_target"]

dt["url_target"] = dt["dummy"]

dt.drop(["dummy"], axis=1)

dt.rename({'url_target':'url_source','page_title_target':'url_target','url_source':'page_title_target'}, axis= 1, inplace=True)

dt

df = dt.head(100)

dr=df.drop(["dummy"], axis=1)


#Checking for the current working directory
import os
os. getcwd()

#Export CSV

export_csv = dr.to_csv (r'C:\Users\vamsi\Downloads\crawl.csv', index = None, header=True)
    

865
865
1515
1645
2930
